<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 8**  
## **Современные методы прогнозирования временных рядов: от SOTA-архитектур до промышленного внедрения**

---

### **1. Цель и задачи работы**

**Цель работы** — формирование у обучающегося системного представления о методологии перехода от академических SOTA-моделей к надёжным, интерпретируемым и экономически обоснованным промышленным системам прогнозирования временных рядов. Работа направлена на освоение практических навыков проектирования end-to-end пайплайнов, включающих feature engineering, backtesting с учётом latency, мониторинг дрейфа, оценку по бизнес-метрикам, интерпретацию для нетехнических пользователей и анализ совокупной стоимости владения (TCO).

**Основные задачи работы:**  
1. Использовать обработанный табличный датасет из Практической работы № 1, содержащий не менее 720 последовательных наблюдений, с подтверждённой стационарностью, стабилизированной дисперсией и богатым набором инженерных признаков (лаги, скользящие окна, циклические кодировки), дополненный промышленными артефактами: задержками в поступлении данных, аномалиями и структурными сдвигами.  
2. Формализовать задачу прогнозирования как **промышленную систему**, а не как отдельную модель, с чётким разделением на этапы: подготовка признаков, обучение, serving, мониторинг, переобучение.  
3. Реализовать и сравнить три стратегии построения прогнозной системы:  
   – **Универсальная модель** (global model на всех рядах),  
   – **Персонализированная модель** (индивидуальная настройка на ряд или кластер),  
   – **Гибридная модель** (статистический бенчмарк + коррекция глубокой моделью).  
4. Провести backtesting с учётом **реальных ограничений**: latency (данные за день *t* доступны только на *t+2*), частота переобучения, доступность экзогенных признаков.  
5. Построить и протестировать современные SOTA-модели:  
   – **NeuralForecast**: NHITS, NBEATSx, TiDE,  
   – **Darts**: TFT, Transformer, TCN,  
   – **Meta Kats**: Prophet+LSTM ансамбли, change point detection,  
   – **Amazon Forecast** (через API или эмуляцию),  
   – **Собственные модели** из работ № 3, № 5, № 6 (LightGBM, TimesNet, PatchTST).  
6. Провести углублённую диагностику промышленной адекватности:  
   – Обнаружение **data drift** (PSI, KS-тест по признакам),  
   – Обнаружение **concept drift** (контрольные карты ошибки),  
   – Анализ устойчивости к аномалиям (ошибка при наличии всплесков).  
7. Оценить качество прогнозов не только по MAE/RMSE, но и по **бизнес-функции потерь** (например, cost = 2·underforecast + 1·overforecast) — на независимой тестовой выборке, не участвовавшей в настройке.  
8. Применить **тест Дикболда–Мариано** для статистического сравнения не только точности, но и бизнес-стоимости прогнозов топ-5 систем.  
9. Проанализировать интерпретируемость для бизнеса:  
   – Генерация текстовых объяснений на языке предметной области,  
   – Визуализация вклада компонент (тренд, сезонность, событие),  
   – Использование SHAP или surrogate-моделей для «чёрных ящиков».  
10. Оценить вычислительную и экономическую эффективность:  
    – Время обучения и инференса,  
    – Потребление памяти и GPU-часы,  
    – Совокупная стоимость владения (TCO): вычисления + поддержка + риск.  
11. Разработать минимальный **промышленный стек на open-source**:  
    – Feature store (на основе pandas/Dask),  
    – Обучение и логирование (MLflow),  
    – Serving (FastAPI + ONNX),  
    – Мониторинг (Evidently).  
12. Обеспечить воспроизводимость и открытость: опубликовать код, сравнительные таблицы, Docker-образ и аналитический отчёт.

---

### **2. Теоретические предпосылки**

SOTA-модели, демонстрирующие рекордные результаты на бенчмарках M4/M5, часто оказываются неприменимы в промышленности из-за игнорирования **реальных ограничений**: задержек в данных, необходимости интерпретации, стоимости переобучения и риска отказа. Успешное внедрение требует не просто выбора архитектуры, а проектирования **наблюдаемой, восстанавливаемой и экономически оправданной системы**.

Ключевым условием корректного сравнения является **единый промышленный сценарий**: все модели оцениваются в одинаковых условиях latency, доступности признаков и частоты обновления. Использование «идеального» тестового набора без учёта этих факторов приводит к смещённым выводам.

Backtesting должен имитировать **реальный процесс принятия решений**: прогноз на *t+h* строится только на данных, доступных к моменту *t*. Единственно корректный подход — **rolling origin с учётом latency**, реализуемый через кастомные временные разбиения.

Выбор метрики для оценки также критичен: в промышленности **бизнес-функция потерь** (например, штраф за занижение спроса) важнее MAE. Модель с худшим MAE, но меньшими финансовыми потерями — предпочтительнее.

Интерпретируемость в продакшене — не опция, а требование. Даже глубокая модель должна предоставлять объяснения на языке предметной области: «Прогноз повышен из-за праздника и тренда роста», а не «SHAP-значение признака lag_7 = +12.3».

После развёртывания модель оценивается не один раз, а **непрерывно**: через мониторинг дрейфа данных, качества и стабильности. Это гарантирует долгосрочную надёжность системы.

---

### **3. Порядок выполнения работы**

Работа выполняется в несколько последовательных этапов, каждый из которых направлен на достижение конкретных образовательных и исследовательских результатов.

#### **3.1. Использование данных из Практической работы № 1**

**Задача:** Обеспечить методологическую преемственность и введение промышленных артефактов.  
**Требования к выполнению:**  
– В качестве входных данных используется финальный обработанный датасет из Практической работы № 1:  
  • Объём: не менее 720 последовательных наблюдений;  
  • Наличие целевой переменной и не менее пяти дополнительных признаков (включая лаги \( y_{t-1}, y_{t-7}, y_{t-30} \), скользящие средние, циклические кодировки времени);  
  • Подтверждённая стационарность (результаты ADF/KPSS);  
  • Сохранённые параметры преобразований (например, \( \lambda \) для Бокса–Кокса).  
– Дополнительно вводятся промышленные артефакты:  
  • Задержка в поступлении данных: информация за день *t* доступна только с *t+2*;  
  • Аномалии: искусственные всплески в 2–3 точках последнего года;  
  • Структурный сдвиг: увеличение дисперсии на последних 60 наблюдениях.  
– Все признаки представлены в табличной форме без пропусков на момент обучения.  
– Обратное преобразование прогнозов осуществляется с использованием сохранённых параметров.

#### **3.2. Формализация задачи и разделение данных**

**Задача:** Корректно сформулировать промышленную задачу прогнозирования с учётом latency.  
**Требования к выполнению:**  
– Целевая переменная: \( y_t \); признаки \( X_t \): все доступные лаги, статистики и экзогенные факторы, **доступные к моменту прогноза**.  
– Разделение данных:  
  • Тренировочная + валидационная часть: всё до \( T - 90 \);  
  • Финальный тест: последние 90 наблюдений — **только для итоговой оценки, не участвует в настройке**.  
– Для backtesting: кастомное разбиение с **rolling origin** и **latency window = 2 дня**, без перемешивания, с сохранением порядка.

#### **3.3. Определение стратегий и архитектур**

**Задача:** Привести каждую систему к её оптимальной конфигурации в промышленных условиях.  
**Требования к выполнению:**  
– Реализуются три стратегии:  
  • **Универсальная**: NeuralForecast (NHITS) на всех данных;  
  • **Персонализированная**: LightGBM с индивидуальной настройкой на ряд;  
  • **Гибридная**: Seasonal Naive + коррекция через TimesNet.  
– Для каждой модели определяются гиперпараметры на основе рекомендаций литературы:  
  • NHITS: `n_blocks=[1,1,1], dropout=0.1`;  
  • TFT: `hidden_size=32, attention_head_size=4`;  
  • LightGBM: `learning_rate ∈ [0.01,0.1], max_depth ∈ [3,10]`.  
– Настройка:  
  • RandomizedSearchCV (n_iter=20) с cv=кастомный TimeSeriesSplit с latency;  
  • Целевая метрика: **бизнес-функция потерь** (например, weighted MAE);  
  • Все процедуры выполняются только на тренировочной части.  
– Фиксация: оптимальные гиперпараметры, значение метрики на валидации, время настройки.

#### **3.4. Backtesting с учётом промышленных ограничений**

**Задача:** Реализовать реалистичную оценку качества.  
**Требования к выполнению:**  
– Backtesting выполняется в 5 итераций с rolling origin;  
– На каждой итерации:  
  • Модель обучается на данных до момента *t*;  
  • Прогноз строится на горизонт \( h = 1, 7, 30 \), **используя только признаки, доступные к *t***;  
  • Оценивается MAE, RMSE и **бизнес-функция потерь**;  
– Фиксируется:  
  • Общее время вычислений,  
  • Частота срабатывания тревог (ошибка > порога).

#### **3.5. Построение и настройка моделей**

**Задача:** Реализовать единообразный интерфейс для всех SOTA-моделей.  
**Требования к выполнению:**  
– **NeuralForecast**: `NHITS`, `NBEATSx`, `TiDE`;  
– **Darts**: `TFTModel`, `TransformerModel`, `TCNModel`;  
– **Kats**: `ProphetParams`, `LSTMParams`, `SARIMAXParams`;  
– **Классические модели**: LightGBM, Ridge (из работы № 3);  
– **Глубокие модели**: TimesNet, PatchTST (из работ № 5, № 6);  
– Для всех моделей: прогноз на тесте, обратное преобразование, сохранение времени и метаданных.

#### **3.6. Диагностика промышленной адекватности**

**Задача:** Убедиться, что система устойчива к реальным условиям.  
**Требования к выполнению:**  
– **Data drift**: расчёт PSI (Population Stability Index) для всех признаков;  
– **Concept drift**: построение контрольных карт (control charts) для ошибки;  
– **Устойчивость к аномалиям**: сравнение MAE до и после добавления всплесков;  
– **Latency-корректность**: проверка, что модель не использует «будущие» признаки;  
– Диагностика обязательна для всех топ-5 систем по бизнес-метрике.

#### **3.7. Оценка качества и статистическое сравнение**

**Задача:** Обоснованно сравнить системы по объективным критериям.  
**Требования к выполнению:**  
– Расчёт метрик на финальной тестовой выборке для каждого горизонта: MAE, RMSE, MASE, **бизнес-функция потерь**;  
– Применение **теста Дикболда–Мариано** для попарного сравнения бизнес-стоимости прогнозов топ-5 систем;  
– Составление сравнительных таблиц:  
  • Системы × стратегии × горизонты (метрики, время, TCO);  
  • Интерпретируемость (тип объяснений, язык);  
  • Диагностические показатели (PSI, control chart violations);  
– Ранжирование систем по взвешенной оценке: точность (30%), надёжность (30%), интерпретируемость (20%), TCO (20%).

#### **3.8. Анализ интерпретируемости для бизнеса**

**Задача:** Оценить способность системы объяснять прогноз нетехническим пользователям.  
**Требования к выполнению:**  
– Для LightGBM/Ridge: генерация текстовых объяснений на основе коэффициентов/важности;  
– Для глубоких моделей: использование SHAP или обучение линейного surrogate для генерации объяснений;  
– Формат объяснений:  
  • «Прогноз на 15 июня = 120 (+12% к прошлой неделе) из-за праздника (вклад +8%) и роста тренда (+4%)»;  
– Проведение локальной интерпретации для критических точек;  
– Результаты интерпретации включаются в сравнительные таблицы и демо-интерфейс.

#### **3.9. Анализ вычислительной и экономической эффективности**

**Задача:** Оценить практическую применимость систем в условиях ограниченных ресурсов.  
**Требования к выполнению:**  
– Фиксация времени:  
  • Обучение (в минутах),  
  • Инференс (в миллисекундах на точку);  
– Оценка потребления памяти и GPU (через `nvidia-smi` или `memory_profiler`);  
– Расчёт **TCO**:  
  • Вычислительные затраты (GPU-часы × стоимость),  
  • Затраты на поддержку (оценка сложности пайплайна),  
  • Ожидаемый ущерб от отказа (частота × средний убыток);  
– Включение столбца «Экономическая эффективность» в сравнительные таблицы.

#### **3.10. Разработка промышленного демо-стека**

**Задача:** Создать минимально жизнеспособный production-стек на open-source.  
**Требования к выполнению:**  
– Feature store: генерация признаков через pandas-функции с кэшированием;  
– Обучение и логирование: MLflow с сохранением параметров, метрик, артефактов;  
– Serving: FastAPI-эндпоинт `/forecast` с ONNX-оптимизированной моделью;  
– Мониторинг: Evidently-отчёт по дрейфу признаков и ошибке;  
– Dockerfile для полной воспроизводимости окружения.

#### **3.11. Публикация результатов исследования**

**Задача:** Обеспечить открытость и воспроизводимость работы.  
**Требования к выполнению:**  
– Для кода и результатов составляется метаописание, включающее:  
  • Список систем и стратегий;  
  • Оптимальные гиперпараметры и метрики на валидации;  
  • Сравнительные таблицы по всем системам, стратегиям и горизонтам;  
  • Ключевые выводы и рекомендации по внедрению;  
  • Инструкцию по запуску демо-стека;  
– Все материалы публикуются в публичном репозитории (GitHub/GitLab);  
– Docker-образ публикуется в GitHub Container Registry или Docker Hub;  
– Использование открытых лицензий: MIT (код), CC BY 4.0 (данные).

---

### **4. Дополнительные исследовательские задания**

1. Сравнение open-source (NeuralForecast) и облачных решений (Amazon Forecast) по качеству и стоимости.  
2. Анализ отказоустойчивости: поведение системы при пропуске цикла переобучения.  
3. Реализация human-in-the-loop: механизм ручной коррекции прогноза и его учёт в следующем обучении.  
4. Прогнозирование с неопределённостью: сравнение конформного прогнозирования и квантильных моделей.  
5. Оценка энергоэффективности: расчёт CO₂-следа для разных архитектур (LightGBM vs. NHITS).  
6. Исследование зависимости TCO от частоты переобучения (ежедневно, еженедельно, ежемесячно).

---

### **5. Требования к отчёту**

Отчёт оформляется в соответствии с ГОСТ 7.32–2017 и должен содержать следующие разделы:  
1. **Введение** — разрыв между SOTA и production, актуальность промышленного ML, обзор литературы, указание на преемственность с Практическими работами № 1–№ 7.  
2. **Методология** — описание данных, промышленного сценария, стратегий, архитектур, backtesting с latency, бизнес-функции потерь.  
3. **Результаты экспериментов** — сравнительные таблицы, графики backtesting, результаты теста Дикболда–Мариано, диагностика дрейфа, примеры бизнес-объяснений.  
4. **Обсуждение** — анализ TCO, сравнение стратегий, роль интерпретации, соответствие выводов реальным кейсам (Amazon, Walmart, Uber).  
5. **Заключение** — итоговые выводы, практические рекомендации по выбору стратегии внедрения, ограничения исследования.  
6. **Список использованных источников** — оформлен в соответствии с ГОСТ Р 7.0.5–2008.  
7. **Приложения** — фрагменты кода, скриншоты демо-стека, примеры сравнительных таблиц, архитектура пайплайна.  
Отчёт сопровождается ссылками на репозиторий с кодом, Docker-образ и опубликованные данные.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Полное выполнение всех этапов; backtesting с latency; диагностика дрейфа; оценка по бизнес-метрике; тест Дикболда–Мариано; бизнес-интерпретация; анализ TCO; функциональный демо-стек (MLflow, FastAPI, Evidently); публикация всех материалов; отчёт, соответствующий ГОСТ. |
| **Хорошо** | Выполнение основных этапов; backtesting с учётом latency; наличие сравнительных таблиц и базовой диагностики; реализация основных систем; отсутствие лишь отдельных элементов (например, TCO или демо-стека). |
| **Удовлетворительно** | Реализованы 3–4 системы без полной диагностики; есть сравнение по MAE; отсутствует бизнес-метрика, интерпретация или мониторинг. |
| **Неудовлетворительно** | Отсутствуют ключевые компоненты работы (данные из работы № 1 не использованы, нет backtesting с latency, отсутствует сравнительный анализ, отчёт не представлен). |
